In [1]:
import graph_tool as gt

In [2]:
from Scripts.Individual import Individual
import numpy as np
from random import sample

In [3]:
from graph_tool.generation import price_network

In [4]:
G = price_network(N = 5000, gamma = 1, directed = False)

In [5]:
G.num_edges()

4999

__Attributing an "Individual" object to a vertex.__

In [8]:
ind_vertex_property = G.new_vertex_property("object")
ind = np.asarray([Individual(5) for n in range(5000)])

In [ ]:
ind_vertex_property

In [ ]:
N = 500
alpha = 0.1
omega = 0.2
indices = np.asarray(sample(list(range(N)), k = N))
positive_tendency = indices[:int(alpha*N)]
negative_tendency = indices[int(alpha*N):int(alpha*N) + int(omega*N)]
neutral_tendency  = indices[int(alpha*N) + int(omega*N):]

vertices = np.zeros(N)
vertices[positive_tendency] = 1
vertices[negative_tendency] = -1
vertices[neutral_tendency]  = 0

ind_vertex_property.a = vertices

In [ ]:
ind_vertex_property[15]

0

__Iterating through edge__

In [ ]:
from Scripts.Entropy import JSD

In [10]:
edges = G.get_edges()

In [ ]:
def get_ind(edge: np.array):
    u = ind[edge[0]]
    v = ind[edge[1]]
    
    distance = 1 - JSD(u.P, v.P)
    
    return distance

In [ ]:
get_ind(edges[0])

0.8893481864237773

In [ ]:
np.apply_along_axis(get_ind, 1, edges)

array([0.88934819, 0.93307606, 0.94415532, ..., 0.8768589 , 0.94256373,
       0.93318595])

In [ ]:
distance_property = G.new_edge_property("double")
distance_property.a = np.apply_along_axis(get_ind, 1, G.get_edges())

In [153]:
distance_property[G.edges().next()]

0.8893481864237773

In [192]:
def _compute_sigma_attribute(node: int) -> float:
    neigh_info = G.get_all_edges(node, eprops = [distance_property])
    return neigh_info.T[2].sum()

In [157]:
neighbour_prop = G.get_all_edges(0, eprops = [distance_property])
print(neighbour_prop)

[[0.00000000e+00 1.00000000e+00 8.89348186e-01]
 [0.00000000e+00 1.40000000e+02 9.09706309e-01]
 [0.00000000e+00 3.48000000e+02 9.04875937e-01]
 [0.00000000e+00 5.54000000e+02 8.90142171e-01]
 [0.00000000e+00 1.28700000e+03 9.23974347e-01]
 [0.00000000e+00 1.55600000e+03 8.46857398e-01]
 [0.00000000e+00 1.57800000e+03 9.54637148e-01]
 [0.00000000e+00 1.69500000e+03 8.41329600e-01]
 [0.00000000e+00 2.25000000e+03 8.43230647e-01]
 [0.00000000e+00 3.38000000e+03 9.16913045e-01]
 [0.00000000e+00 4.20800000e+03 8.92157273e-01]
 [0.00000000e+00 4.98300000e+03 8.40456709e-01]]


In [209]:
f = lambda x: G.get_total_degrees(G.get_vertices())[x]**2
neigh = G.get_all_neighbors(0)
np.append(f(neigh), -15).min()

-15.0

In [182]:
vertices.reshape(1, -1)

array([[   0,    1,    2, ..., 4997, 4998, 4999]])

In [189]:
np.expand_dims(vertices, 1)

array([[   0],
       [   1],
       [   2],
       ...,
       [4997],
       [4998],
       [4999]])

In [193]:
vertices = G.get_vertices()
v = np.expand_dims(vertices, 1)
a = np.apply_along_axis(_compute_sigma_attribute, 1, v)

In [197]:
distance_property.a.sum()

PropertyArray(4431.37964658)

array([ 12, 142,  44, ...,   1,   1,   1], dtype=uint64)

In [13]:
def get_ind_objects(edge: np.array):
    return [ind[edge[0]], ind[edge[1]]]

In [14]:
edges = G.get_edges()

In [18]:
for k, u, v in list(enumerate(np.apply_along_axis(get_ind_objects, 1, edges)))[:5]:
    print(k, u, v)

ValueError: not enough values to unpack (expected 3, got 2)

In [26]:
edges[0][::-1]

array([0, 1])

In [25]:
edges[0]

array([1, 0])